<a href="https://colab.research.google.com/github/shailendrabhandari/ACIT4630_Advanced_MLandDL/blob/master/EEG-Siamnse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Files

Here we read the whole data files names

In [ ]:
import glob, os
train_normal_files = []
for file in glob.glob("/Users/mohamedr/projects/rules/nmt_scalp_eeg_dataset/normal/train/*edf"):#*/*/*.edf"):
    train_normal_files.append(file)
    #epilespy_files.extend(files)
print(len(train_normal_files))

train_abnormal_files = []
for file in glob.glob("/Users/mohamedr/projects/rules/nmt_scalp_eeg_dataset/abnormal/train/*edf"):#*/*/*.edf"):
    train_abnormal_files.append(file)
    #epilespy_files.extend(files)
print(len(train_abnormal_files))

test_normal_files = []
for file in glob.glob("/Users/mohamedr/projects/rules/nmt_scalp_eeg_dataset/normal/eval/*edf"):#*/*/*.edf"):
    test_normal_files.append(file)
    #epilespy_files.extend(files)
print(len(test_normal_files))

test_abnormal_files = []
for file in glob.glob("/Users/mohamedr/projects/rules/nmt_scalp_eeg_dataset/abnormal/eval/*edf"):#*/*/*.edf"):
    test_abnormal_files.append(file)
    #epilespy_files.extend(files)
print(len(test_abnormal_files))

1877
355
95
90


## Reading Data

This function is used to read the whole data to numpy arrays

In [ ]:
%%capture
#NMT dataset
import mne
from tqdm import tqdm
import random
import numpy as np

def build_data(raw_data):
    
    eeg_data = []
    for file in raw_data:
        data = mne.io.read_raw_edf(file, verbose=False, preload=True)
        try:
            if data.last_samp > 845:
                data.filter(l_freq=1, h_freq=45, verbose=False)
                data = mne.make_fixed_length_epochs(data, duration=1, overlap=0, verbose=False)
                data = data.get_data()
                eeg_data.append(data)
        except:
            pass
    eeg_data = np.vstack(eeg_data)
    return eeg_data

np.random.seed(42)

train_normal_files = random.sample(train_normal_files, 100)
train_abnormal_files = random.sample(train_abnormal_files, 100)

train_normal_data = build_data(train_normal_files)
train_abnormal_data = build_data(train_abnormal_files)
test_normal_data = build_data(test_normal_files)
test_abnormal_data = build_data(test_abnormal_files)

In [ ]:
train_normal_data.shape, train_abnormal_data.shape, test_normal_data.shape, test_abnormal_data.shape

((70819, 21, 200), (76455, 21, 200), (67090, 21, 200), (64794, 21, 200))

The next functions are helper functions to standerdize the data and calculate accuracies

In [ ]:
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)


def evaluate_model(model, loss_func, data_iter):
    model.eval()
    loss_sum, n = 0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x)
            y_pred = y_pred.squeeze()
            loss = loss_func(y_pred,y)
            loss_sum += loss.item()
            n += 1
        return loss_sum / n


def cal_accuracy(model, data_iter):
    ytrue = []
    ypreds = []
    with torch.no_grad():
        for x, y in data_iter:
            yhat = model(x)
            yhat = [0 if i<0.5 else 1 for i in yhat]
            ytrue.extend(list(y.numpy()))
            ypreds.extend(yhat)

    return (accuracy_score(ytrue, ypreds), 
            confusion_matrix(ytrue, ypreds), 
            precision_score(ytrue, ypreds), 
            recall_score(ytrue, ypreds),
            f1_score(ytrue, ypreds))

def standardize_data(train_features, test_features):
    scaler = StandardScaler3D()
    train_features = scaler.fit_transform(train_features)
    #val_features = scaler.fit_transform(val_features)
    test_features = scaler.transform(test_features)
    return train_features, test_features


def data_loader(features, labels, device, batch_size, shuffle=True):

    features = torch.Tensor(features).float().to(device)
    labels = torch.Tensor(labels).float().to(device)
    data = torch.utils.data.TensorDataset(features, labels)
    data_iter = torch.utils.data.DataLoader(data, batch_size, shuffle=shuffle)

    return data_iter

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The next cells to standardize the data, build the labels arrays and build arrays for the features and the labels

In [ ]:
# Standardize the data
train_normal_data, test_normal_data = standardize_data(train_normal_data, test_normal_data)
train_abnormal_data, test_abnormal_data = standardize_data(train_abnormal_data, test_abnormal_data)

In [ ]:
# building data labels for the sliding windows of the data 
test_abnormal_labels=[1 for x in test_abnormal_data]
test_normal_labels=[0 for x in test_normal_data]
train_abnormal_labels=[1 for x in train_abnormal_data]
train_normal_labels=[0 for x in train_normal_data]

test_abnormal_labels = np.array(test_abnormal_labels)
test_normal_labels = np.array(test_normal_labels)
train_abnormal_labels = np.array(train_abnormal_labels)
train_normal_labels = np.array(train_normal_labels)

In [ ]:
train_abnormal_data.shape, train_abnormal_labels.shape

((76455, 21, 200), (76455,))

In [ ]:
train_features = np.concatenate((train_normal_data, train_abnormal_data))
test_features = np.concatenate((test_normal_data, test_abnormal_data)) 
train_labels = np.concatenate((train_normal_labels, train_abnormal_labels)) 
test_labels = np.concatenate((test_normal_labels, test_abnormal_labels)) 
train_features.shape, train_labels.shape

((147274, 21, 200), (147274,))

## Stacked LSTM

Here is the model we used for training on the whole data.

Remember as we have the shape of the data as (#num of samples, #num of channels, #number of time points), The LSTM network specs is as follows:
- input_size = 200 (num of time points)
- hidden_units = number of LSTMs, this can be changed (16, 64 ...etc)
- num_layers = number of LSTM layers to stack (number of channels in EEG (21))

The LSTM layers are followed by linear layer and then flattened before the sigmoid function to generate outputs.

In [ ]:
from tqdm import tqdm
import torch.nn as nn
import torch
from torch.autograd import Variable

class StackedLSTM(nn.Module):
    def __init__(self, input_size, num_channels, hidden_units):
        super().__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=num_channels,
            bidirectional=True
        )

        self.linear = nn.Linear(in_features=hidden_units, out_features=1)

    def forward(self, x):
        lstm_out, (hn, _) = self.lstm(x)
        out = self.linear(hn[0]).flatten()
        out = torch.sigmoid(out)
        return out

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn

BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE, shuffle=True)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 21
model=StackedLSTM(200, 21, 64)
model.to(DEVICE)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    train_accuracy = cal_accuracy(model, train_iter)
    val_accuracy = cal_accuracy(model, test_iter)
    
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        train_accuracy[0], ",F1: ", 
        train_accuracy[4], ",Precision: ", 
        train_accuracy[2], ",Recall: ", 
        train_accuracy[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        val_accuracy[0], ",F1: ", 
        val_accuracy[4], ",Precision: ", 
        val_accuracy[2], ",Recall: ", 
        val_accuracy[3])

Data Loader....


NameError: name 'train_features' is not defined

It is suitable here to view the results in graphs, chanegs of loss function over time and changes in accuracies too. 

In [ ]:
val_accuracy

(0.7424479087683116,
 array([[50158, 16932],
        [17035, 47759]]),
 0.7382634369541358,
 0.7370898539988271,
 0.7376761787079585)

Until now, we were just training on large subsets of the data which achieved this accuracies.

# Working with 3 data samples

In [ ]:
%%capture
np.random.seed(42)

train_normal_files = random.sample(train_normal_files, 3)
train_abnormal_files = random.sample(train_abnormal_files, 3)

train_normal_data = build_data(train_normal_files)
train_abnormal_data = build_data(train_abnormal_files)
test_normal_data = build_data(test_normal_files)
test_abnormal_data = build_data(test_abnormal_files)

In [ ]:
# Standardize the data
train_normal_data, test_normal_data = standardize_data(train_normal_data, test_normal_data)
train_abnormal_data, test_abnormal_data = standardize_data(train_abnormal_data, test_abnormal_data)

In [ ]:
# building data labels for the sliding windows of the data 
test_abnormal_labels=[1 for x in test_abnormal_data]
test_normal_labels=[0 for x in test_normal_data]
train_abnormal_labels=[1 for x in train_abnormal_data]
train_normal_labels=[0 for x in train_normal_data]

test_abnormal_labels = np.array(test_abnormal_labels)
test_normal_labels = np.array(test_normal_labels)
train_abnormal_labels = np.array(train_abnormal_labels)
train_normal_labels = np.array(train_normal_labels)

In [ ]:
train_features = np.concatenate((train_normal_data, train_abnormal_data))
test_features = np.concatenate((test_normal_data, test_abnormal_data)) 
train_labels = np.concatenate((train_normal_labels, train_abnormal_labels)) 
test_labels = np.concatenate((test_normal_labels, test_abnormal_labels)) 
train_features.shape, train_labels.shape

((4007, 21, 200), (4007,))

In [ ]:
test_features.shape, test_labels.shape

((131884, 21, 200), (131884,))

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn

BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE, shuffle=True)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 21
model=StackedLSTM(200, 21, 64)
model.to(DEVICE)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    train_accuracy = cal_accuracy(model, train_iter)
    
    # This is a bad practice, using test data for validation
    # need to fix it
    val_accuracy = cal_accuracy(model, test_iter)
    
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        train_accuracy[0], ",F1: ", 
        train_accuracy[4], ",Precision: ", 
        train_accuracy[2], ",Recall: ", 
        train_accuracy[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        val_accuracy[0], ",F1: ", 
        val_accuracy[4], ",Precision: ", 
        val_accuracy[2], ",Recall: ", 
        val_accuracy[3])

Data Loader....
Training Model....
Epoch 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:56<00:00,  1.77s/it]


Train loss: 0.6851712372153997 ,Train Accuracy:  0.6181682056401298 ,F1:  0.3277680140597539 ,Precision:  0.7803347280334728 ,Recall:  0.20745272525027808
Val loss: 0.6921455461231976 , Val Accuracy:  0.5148918746777471 ,F1:  0.1870234827691369 ,Precision:  0.5293482952093224 ,Recall:  0.11357533104917122
Epoch 2


 25%|█████████████████████████                                                                           | 8/32 [00:16<00:49,  2.05s/it]


KeyboardInterrupt: 

In [ ]:
val_accuracy

(0.5148918746777471,
 array([[60547,  6543],
        [57435,  7359]]),
 0.5293482952093224,
 0.11357533104917122,
 0.1870234827691369)

# Siamese Network

The next cell is LSTM-Siamese Network. The network aims to get two latent represnatations of the two samples using the LSTM we used earlier

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.autograd import Variable

class Siamese(nn.Module):
    def __init__(self, input_size, num_channels, hidden_units):
        super(Siamese, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=num_channels,
            bidirectional=True
        )
        self.linear = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.out = nn.Linear(in_features=hidden_units, out_features=1)
        
        self.normal_class_representation = torch.zeros(hidden_units)
        self.abnormal_class_representation = torch.zeros(hidden_units)
        
    def forward_one(self, x):
        lstm_out, (hn, _) = self.lstm(x)
        x = self.linear(hn[0])
        return x

    def forward(self, x):
        anchor, pos, neg = x[:, 0, :, :], x[:, 1, :, :], x[:, 2, :, :]
        anchor = self.forward_one(anchor)
        pos = self.forward_one(pos)
        neg = self.forward_one(neg)
        #dis = torch.abs(out1 - out2)
        #out = self.out(dis)
        #out = torch.sigmoid(out)
        return anchor, pos, neg

if __name__ == '__main__':
    net = Siamese(200, 21, 64)
    # 200 is number of time points
    # 21 number of channels
    # 64 number of LTSM units
    
net

Siamese(
  (lstm): LSTM(200, 64, num_layers=21, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=64, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
x1 = torch.Tensor(1, 21, 200)
x2 = torch.Tensor(1, 21, 200)
x3 = torch.Tensor(1, 21, 200)

anchor, pos, neg = net(x1, x2, x3)
output = triplet_loss(anchor, pos, neg)
output

TypeError: Siamese.forward() takes 2 positional arguments but 4 were given

In [ ]:
%%capture
np.random.seed(42)
#build data
train_normal_files = random.sample(train_normal_files, 5)
train_abnormal_files = random.sample(train_abnormal_files, 5)
train_normal_data = build_data(train_normal_files)
train_abnormal_data = build_data(train_abnormal_files)
test_normal_data = build_data(test_normal_files)
test_abnormal_data = build_data(test_abnormal_files)

In [ ]:
# Standardize the data
train_normal_data, test_normal_data = standardize_data(train_normal_data, test_normal_data)
train_abnormal_data, test_abnormal_data = standardize_data(train_abnormal_data, test_abnormal_data)

In [ ]:
x = np.random.rand(4, 1, 21, 200)
#print(x.shape)
x = np.repeat(x, 3, axis=1)
x.shape

(4, 1, 21, 200)


(4, 3, 21, 200)

In [ ]:
test_labels = [0 for i in test_normal_data]+[1 for i in test_abnormal_data]
test_labels = np.array(test_labels)
test_data = np.concatenate((test_normal_data, test_abnormal_data))
test_data = test_data.reshape(131884,1,  21, 200)
test_data = np.repeat(test_data, 3, axis=1)
test_data.shape, test_labels.shape

((131884, 3, 21, 200), (131884,))

In [ ]:
train_data = []
train_labels = []
train_features = np.concatenate((train_normal_data, train_abnormal_data))

for i in range(4000):
    anchor = random.sample(range(train_features.shape[0]), 1)
    anchor = anchor[0]
    if anchor > len(train_normal_data):
        anchor = train_features[anchor]
        pos = random.sample(range(len(train_abnormal_data)), 1)
        pos = pos[0]
        pos = train_abnormal_data[pos]
        neg = random.sample(range(len(train_normal_data)), 1)
        neg = neg[0]
        neg = train_normal_data[neg]
        train_data.append([anchor, pos, neg]) 
        train_labels.append(1)
    else:
        anchor = train_features[anchor]
        pos = random.sample(range(len(train_normal_data)), 1)
        pos = pos[0]
        pos = train_normal_data[pos]
        neg = random.sample(range(len(train_abnormal_data)), 1)
        neg = neg[0]
        neg = train_abnormal_data[neg]
        train_data.append([anchor, pos, neg]) 
        train_labels.append(0) #0 is label
    
train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [ ]:
train_data.shape, train_labels.shape

((4000, 3, 21, 200), (4000,))

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn

BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 10
    
print("Data Loader....")
train_iter = data_loader(train_data, train_labels, DEVICE, BATCH_SIZE, shuffle=True)
test_iter = data_loader(test_data, test_labels, DEVICE, BATCH_SIZE, shuffle=True)
    
print("Training Model....")
n_chans = 21
model=Siamese(200, 21, 64)
model.to(DEVICE)
loss_func = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    num_y = 0
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y)  in enumerate(tqdm(train_iter)):
        anchor, pos, neg = model(x)
        #y_pred = y_pred.squeeze()
        loss = loss_func(anchor, pos, neg)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        
        #Here we save class representation within the model
        if epoch == NUM_EPOCHS:
            for i, lab in enumerate(list(y)):
                num_y += 1 
                if lab == 0:
                    model.normal_class_representation += pos[i]
                    model.abnormal_class_representation += neg[i]
                else:
                    model.normal_class_representation += neg[i]
                    model.abnormal_class_representation += pos[i]
            model.normal_class_representation = model.normal_class_representation / num_y
            model.abnormal_class_representation = model.abnormal_class_representation / num_y
                                
    print(loss_sum)
    #val_loss = evaluate_model(model, loss_func, test_iter)
    #train_accuracy = cal_accuracy(model, train_iter)
    
    # This is a bad practice, using test data for validation
    # need to fix it
    #val_accuracy = cal_accuracy(model, test_iter)
    
    #print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
    #    train_accuracy[0], ",F1: ", 
    #    train_accuracy[4], ",Precision: ", 
    #    train_accuracy[2], ",Recall: ", 
    #    train_accuracy[3])
    #print("Val loss:", val_loss, ", Val Accuracy: ", 
    #    val_accuracy[0], ",F1: ", 
    #    val_accuracy[4], ",Precision: ", 
    #    val_accuracy[2], ",Recall: ", 
    #    val_accuracy[3])

Data Loader....
Training Model....
Epoch 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:55<00:00,  5.47s/it]


31.357930719852448
Epoch 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:49<00:00,  5.30s/it]


29.299343585968018
Epoch 3


...

3.052323017269373
Epoch 9


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [03:57<00:00,  7.42s/it]


2.0050703454762697
Epoch 10


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [03:44<00:00,  7.03s/it]

1.6373741645365953


In [ ]:
def cal_accuracy(model, data_iter):
    ytrue = []
    ypreds = []
    with torch.no_grad():
        for x, y in tqdm(data_iter):
            yhat = model(x)
            anchor, pos, neg = model(x)
            anchor = anchor.detach().numpy()
            pos = pos.detach().numpy()
            neg = neg.detach().numpy()
            
            dist2pos = np.linalg.norm(anchor - pos)
            dist2neg = np.linalg.norm(anchor - neg)
            
            yhat = [1 if dist2pos <= dist2neg else 0]
            ytrue.extend(list(y.numpy()))
            ypreds.extend(yhat)
    
    return (accuracy_score(ytrue, ypreds), 
            confusion_matrix(ytrue, ypreds), 
            precision_score(ytrue, ypreds), 
            recall_score(ytrue, ypreds),
            f1_score(ytrue, ypreds))

In [ ]:
test_iter = data_loader(test_data, test_labels, DEVICE, BATCH_SIZE, shuffle=True)

In [ ]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in tqdm(test_iter):
        anchor, pos, neg = model(x)
        anchor = anchor.detach().numpy()
        abnormal_rep =  model.abnormal_class_representation.detach().numpy()
        normal_rep =  model.normal_class_representation.detach().numpy()
        
        ytrue.extend(y)
        for anchor_sample in anchor:
            dist2normal = np.linalg.norm(anchor_sample - normal_rep)
            dist2abnormal = np.linalg.norm(anchor_sample - abnormal_rep)
            if dist2normal >= dist2abnormal:
                yhat = 1
            else:
                yhat = 0
            ypreds.append(yhat)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1031/1031 [52:12<00:00,  3.04s/it]


In [ ]:
accuracy_score(ytrue, ypreds), confusion_matrix(ytrue, ypreds)

(0.7584240696369536,
 array([[36472, 30618],
        [ 1242, 63552]]))